In [25]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import random
import json
from datetime import datetime, timedelta
from utils.generation_utils import generate_prompt, label2email

In [ ]:
# Load the environment variables from the .env file
# In this .env, it contains openai's API Key.
load_dotenv()
# Load the key to call the client.
client = OpenAI()
model_name="gpt-4o-mini"
# model_name="gpt-4o"

In [30]:
pseudo_labels = "./data/pseudo_email_labels_refined_manually.json"
with open(pseudo_labels, 'r') as file:
    data = json.load(file)

# Now 'data' holds the content of the JSON file as a Python dictionary
print(data[0])

{'Spam': 'No', 'Subject': 'Briefint session changed to 9:30 this morning', 'Sender': "Mia O'Connor", 'send_date': '2024-11-11', 'Time_Sensitive': 'Yes', 'Start': '2024-11-11 09:30', 'End': '2024-11-11 10:00', 'Type': 'Event', 'Category': 'Work', 'Format': 'Online', 'Location': 'Microsoft Teams', 'Action_Required': 'Yes', 'Priority_Level': 'Urgent'}


In [31]:
label2email(data[0], client, temperature=0.7, model=model_name)

'From: Mia O’Connor  \nDate: November 11, 2024  \nSubject: Briefing Session Changed to 9:30 This Morning  \n\nDear Team,\n\nI hope this message reaches you in time. Please note that the briefing session originally scheduled for today has been adjusted. We will now meet online from 9:30 AM to 10:00 AM, via Microsoft Teams.\n\nTo ensure a seamless session, please confirm your attendance at your earliest convenience. Should you have any inquiries or require assistance with accessing the session, feel free to reach out.\n\nThank you for your prompt attention to this change.\n\nBest regards,  \nMia O’Connor  '

In [32]:
# Define the input and output paths
output_email_dir = "./data/pseudo_emails"  # Directory to store pseudo email files
output_json_path = f"./data/PAIRED_{model_name}_pseudo_emails.json"  # Final JSON file with three entries
# data_test = data[:3]

In [33]:
paired_json = []
for i in range(len(data[:3])):
    email_file_path = os.path.join(output_email_dir, f"{model_name}_{i+1}.txt")
    
    # content = generate_email(data[i])
    content = label2email(data[i], client, temperature=1.0, model=model_name)
    with open(email_file_path, "w") as email_file:
        email_file.write(content)
    
    paired_entry = {
        "path": f"./pseudo_emails/{model_name}_{i+1}.txt",
        "content": content,
        "label": data[i]
    }
    paired_json.append(paired_entry)

with open(output_json_path, "w") as output_file:
    json.dump(paired_json, output_file, indent=4)